## A tool for calculating the binding energy of a system of objects

In this notebook, I will demonstrate how I developed a solution to calculate the binding energy of 3 objects for a given geometry.

The solution involved:

- Reading input data from a text file
- Checking that the input contains a reasonable number of items
- Calculating binding energies for individual pairwise interactions
- Summing up the individual binding energies to calculate the total binding energy of the system
- Outputting the results with appropriate messages and some error handling

I broke the solution down into separate stages and created a function for each stage. 

The reason I did it this way was to improve: 

- Code readability
- Modularity
- Maintainability
- Reusability 

By improving readability by using functions for each stage, the code becomes easier to maintain. It also allows for reusability of functions in instances where the code might need to be modified for a different use case. It also means that the code is easier to test and debug.

In [1]:
# Date created: 04/04/2024
# Date modified: 16/04/2024
# Created by: Joel Miller
# This is a project to calculate the binding energy of "n" number of objects for a given geometry.
import math
import sys


In stage 1 we are simply defining constants; therfore, it is not neccessary to create any functions. 

In [2]:
# Stage 1

# Define constants
sigma = 3.41e-10  # in meters
epsilon = 1.65e-21  # in joules

# Example case
r_example = 6.82e-10  # in metres
u_example = -1.0e-22  # in Joules


In [3]:
# Stage 2 - Open file containing distances and convert to list of floats

def txt_2_list(txt_file):
    f = open(txt_file, 'r')
    f_list_str = f.readlines()
    f.close()
    n_distances = len(f_list_str)
    r_list = []
    for i in range(n_distances):
        r_list.append(float(f_list_str[i]))
    return r_list
    

An alternative approach to stage 2 would be to manually create a list containing each of the 3 distances. 
If you will only be using this code once, then this approach would be acceptable; however, if you intend to use the code with a different set of distances, then a more universal approach, such as the one above, is more suitable and will help to improve efficiency when handling larger volumes of data.  

From this list, we could create a class to represent pairings between the objects and create a method for calculating the binding energy between specific pairs of objects. 

It is worth noting that at some point in the future we may end up dealing with objects and their positions((x,y) coordinates), rather than the pairwise distances. In this instance, we could create a class of objects with attributes relating to position. We could then create a method for calculating the distances between each of the pairs.

In this stage we open the file containing the distances between each of the 3 pairs of objects. We then read the lines of the file and 



In [4]:
# Stage 3 - Create a function to calculate the binding energy of a single object pairing
def binding_energy_calculator(r):
    return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)
    

In stage 4 we will check that our input list contains a reasonable number of elements by calculating the number of objects from the number of pairwise interactions. 

We know that the number of pairings for $n$ objects is given by:  

$$n_{pairings} = \frac{n(n-1)}{2}$$

Using this, we can rearrange for $n$, which gives the quadratic equation: 

$$n^2 − n − 2n_{pairings} = 0,$$ 

which can be solved using the quadratic formula.

This gives $a = 1,$  $b = -1,$ $c = -2n_{pairings}$.

$$n = \frac {-b \pm \sqrt{b^2 -4ac}} {2a},$$


By checking that the discriminant is an integer we can confirm that our list contains a reasonable number of elements. 

An additional bonus is that we can calculate the number of objects that make up our pairings; this is potentially useful to know for future work.


If we are given a list with an incorrect number of separations, then our calculation for the total binding energy
will be incorrect. For instance, 10 objects provide 27 pairings. If we have a list containing 28 pairings,
then we end up with a non-integer number of objects, which is impossible. We can use this information to include a
check within our code, that tells us if we have a reasonable number of elements in our list. Of course, it's possible
that we randomly end up with a list containing a reasonable number of elements; however, this check minimises the
chance of the code outputting the incorrect result.


In [5]:
# Stage 4 - Check that the list contains a reasonable number of elements

def calc_n_objects(distances):  # Function to calculate the number of objects from a list of separations
    n_pairings = len(distances)  # Find n_pairings from distances list
    a = 1
    b = -1
    c = -2 * n_pairings

    discriminant = (b ** 2) - (4 * a * c)  # calculate the discriminant from the quadratic equation

    # Check to see if the discriminant is a perfect square, if not, then we have a problem with the list of distances
    if math.sqrt(discriminant) % 1 == 0:
        print('The discriminant is a perfect square and we can proceed in calculating the number of objects from our '
              'distances list.')
    else:
        sys.exit('The discriminant is not a perfect square and the distances list is missing data, please '
                 'check you are using the correct data set!')


    # Calculate the roots to find the number of objects 

    root_1 = (-b + math.sqrt(discriminant)) / 2  # This is the root that gives the original number of objects that make up the pairings
    # root_2 = (-b - math.sqrt(discriminant)) / 2  # We can ignore root_2 since it will always be negative
    if n_pairings <= 1:
        print('Given the number of separations in your list, we can determine that there are', int(root_1),
              'objects that make up your', n_pairings, 'pairing.')
    else:
        print('Given the number of separations in your list, we can determine that there are', int(root_1),
              'objects that make up your', n_pairings, 'pairings.')

    return root_1 

In [6]:
# Stage 5 - Calculate the binding energies

def total_binding_energy_calculator(
        distances):  # Function to calculate total binding energy from list of object separations
    binding_energies_list = []  # Initialize empty list to store calculate individual pairwise binding energies

    for r in distances:
        binding_energy = binding_energy_calculator(r)  # calculate individual binding energies using custom function
        binding_energies_list.append(binding_energy)

    total_binding_energy = sum(binding_energies_list)

    n_pairings = len(distances)
    if n_pairings <= 1:
        # print('There is', n_pairings, 'separation in this list.')
        print('The total binding energy of the', n_pairings, 'pairing is', format(total_binding_energy, '.3g'), 'J.')
    else:
        # print('There are', n_pairings, 'separations in this list.')
        print('The total binding energy of the', n_pairings, 'pairings is', format(total_binding_energy, '.3g'), 'J.')

    difference = abs(u_example - binding_energy_calculator(r_example))  # Compare known energy at r_example to calculated energy
    # print('the difference between the example and the calculated result is,', difference, 'Joules')

    if difference < 1e-23:  # If the difference is less than the tolerance, then consider the result to be correct
        print('The example conditions continue to give the correct results; therefore, the code is working '
              'and you can trust these results.')
    else:
        print('The code is broken and these results can\'t be trusted, please consult the developers')

    return total_binding_energy


In [7]:
# Stage 6 - Combine functions to create one
def binding_energy_tool(txt_file):
    distances_list = txt_2_list(txt_file)
    calc_n_objects(distances_list)
    return total_binding_energy_calculator(distances_list)


total_bindin_energy = binding_energy_tool('distances.txt')

The discriminant is a perfect square and we can proceed in calculating the number of objects from our distances list.
Given the number of separations in your list, we can determine that there are 3 objects that make up your 3 pairings.
The total binding energy of the 3 pairings is 3.82e-18 J.
The example conditions continue to give the correct results; therefore, the code is working and you can trust these results.


# Considerations: 

### Who should be involved in developing the final solution?
The development of the final solution could involve: 

- Various stakeholders, depending on their technical expertise or understanding
- Experts (such as physicists or scientists familiar with the problem)
- End users or stakeholders who will use the solution
- Other software engineers who are helping to develop the solution

Collaboration between different roles ensures that the solution meets both technical requirements and user needs.

### How has what you don’t know about the problem impacted your solution?

What don't I know:

- Lacking information about the input data. Is this just a small sample of data, or is it the full set?
- If this isn't the full set, then what format will the full set take on? .csv, .txt etc
- Is this going to be part of a larger project?
- Will the problem involve positions of objects at some point?
- Not knowing how the output will be used 

How has this impacted my solution?
- These unknowns have led me to futureproof my solution. 
- For example, reading in a list of distances, as opposed to including the distances directly in the code. This means that the code will work with any number of pairs of objects.
- Not knowing how the data will be used has meant that I've explicitly stated the information that is being calculated. It means that anyone running the code can easily see what the code is doing without necessarily having to look through the code and read comments. However, this might not be necessary for any future/final projects.

### How might you do it differently in a team or production context?

A team or production context could involve:
- Code reviews to ensure that quality assurance is maintained. Also so other co-developers understand the code, and to help identify bugs
- Team members developing a function each. This could enhance modularity and minimise variable-naming errors
- Setting goals so the whole team has an expectation of where the project should be up to. 
- Regular communication to ensure decisions are informed, perhaps with the use of GitHub.


Additionally, considerations such as error handling, logging, testing, and documentation could be used to ensure the reliability and maintainability of the code.

### What things might you need to consider if this was the start of a larger project?

- Scalability: Will this code be used with more than 3 objects?
- Modularity: Having modular code makes it easier to structure the code and to help with future enhancements. 
- Testing: For a larger project, testing is important to ensure the code functions correctly. Eg, unit tests
- Documentation: Ensuring that the code is easy to use and understand will significantly aid with larger projects
- Version control: To help with effective collaboration and to manage changes to the code. E.g, using git/GitHub 
